# **Giới thiệu**

### Thành viên:

Nhóm `Sinh viên HCMUS`:
- Nguyễn Khắc Vỹ - 19127637
- Nguyễn Phan Vũ - 19127633

### Overview:

`Dataset:` Cơ sở dữ liệu lớn về sách. Sách thuộc nhiều thể loại khác nhau, từ hàng ngàn tác giả khác nhau. Không chỉ tiếng Anh mà còn bao gồm tiếng Ấn Độ, cùng với đó là nhiều thông tin chi tiết khác về từng cuốn sách.

`Competition:` **Xây dựng mô hình dự đoán giá của sách** trên **MachineHack**.

### Features:

**`Title:`** Tên sách

**`Author:`** Tác giả

**`Edition:`** Phiên bản phát hành

**`Reviews:`** Điểm trung bình

**`Ratings:`** Số lượt chấm điểm

**`Synopsis:`** Tóm tắt

**`Genre:`** Thể loại

**`BookCategory:`** Category của sách

**`Price:`** Giá sách

### Data Scale:

**`Training set:`** 6237 records

**`Test set:`** 1560 records

# **Coding**

### Libiraries

In [ ]:
import pandas as pd
import numpy as np
import string 
import os

# Preprocessing 
from googletrans import Translator, constants
from pprint import pprint

from tqdm import tqdm
import time


# NLTK preprocessing
from bs4 import BeautifulSoup
import contractions
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('omw-1.4')

### Data Reading

In [ ]:
train_data_df=pd.read_excel('Data_Train.xlsx')
test_data_df=pd.read_excel('Data_Test.xlsx')

In [ ]:
train_data_df

In [ ]:
print("Tập train: ", train_data_df.shape)
print("Tập test: ", test_data_df.shape)

### Data Preprocessing

#### **`Concat`** 2 tập **`train`** và **`test`** để preprocessing đồng thời

In [ ]:
train_data_df['Set'] = 'train'
test_data_df['Set'] = 'test'

data_df = pd.concat([train_data_df, test_data_df])
data_df = data_df.reset_index(drop=True)
data_df.sample(3)

In [ ]:
print("Dữ liệu sau khi concat: ", data_df.shape)

#### Xử lý từng **`vấn đề`**

##### Cột **`Author`**

Hiện tại, **`Author`** chứa tên tác giả, một cuốn sách có thể được đồng sáng tác bởi nhiều tác giả.

Tuy nhiên, chứa các kí tự ngăn cách giữa các tác giả không đồng nhất => Khiến cho có thể sách do cùng một nhóm tác giả sáng tác, khi count lại được tính là 2 nhóm khác nhau

In [ ]:
def find_unique_punctuations(texts):
  set_of_punctuations = set()
  for text in texts:
    for punc in string.punctuation:
      if punc in text:
        set_of_punctuations.add(punc)
  return set_of_punctuations

In [ ]:
authors = data_df['Author'].copy()

find_unique_punctuations(authors)

Replace các dấu này, về **`','`** cho đồng nhất

In [ ]:
# Định nghĩa một số loại dấu tương ứng với các hành động riêng biệt

# Kí tự không mang quá nhiều ý nghĩa
punctuations_to_remove = ['(',')','!',"'",'.']

# Kí tự '-' thường mang ý nghĩa tương tự dấu cách => thay bằng spacebar
punctuations_to_replace_with_space = ['-']

# Kí tự mang thường ý nghĩa 'and' đồng nhất bằng dấu ','
punctuations_to_replace_with_comma = ['&', '/', ';']

def handle_author_text(text):
   # Makes the text to lowercase
   text = text.lower()

   # Handling each punctuation case 
   for punctuation in punctuations_to_remove:
     text = text.replace(punctuation, '')
   for punctuation in punctuations_to_replace_with_comma:
     text = text.replace(punctuation, ', ')
   for punctuation in punctuations_to_replace_with_space:
     text = text.replace(punctuation, ' ')

   return text

In [ ]:
print("Unique trước khi preprocess:", data_df['Author'].nunique())

authors = authors.apply(handle_author_text)
authors.nunique()
print("Unique trước khi preprocess:", authors.nunique())

###### Thêm cột **`số lượng tác giả`** cho sách

In [ ]:
# Make integers based of different authors
def count_authors(text):
  return text.count(',')+1

num_of_authors = authors.apply(count_authors)
#data = num_of_authors.to_frame()

Đã đồng nhất các giá trị trong cột => Vì số giá trị unique giảm

##### Cột **`Ratings`**

Hiện tại, ratings không chỉ bao gồm số (số lượt rating), mà còn bao gồm text (không cần thiết)

**`Ví dụ:`** 13 customer reviews


In [ ]:
ratings = data_df['Ratings']
ratings[:3]

In [ ]:
ratings = ratings.apply(lambda x: (x.split()[0].replace(',',''))).astype(int)
ratings[:3]

##### Cột **`Reviews`**

In [ ]:
reviews = data_df['Reviews']
reviews[:3]

In [ ]:
reviews = reviews.apply(lambda x: x.split(' ')[0])
reviews = reviews.astype(np.float16)
reviews[:3]

##### Cột **`Title`** và **`Synopsis`**

Phần **`Giới thiệu`** có đề cập dataset có những quyển được gather từ nguồn của Ấn Độ vì vậy **`Synopsis`** không chỉ có mỗi Tiếng Anh

In [ ]:
data_df.head(3)

###### Vấn đề đồng nhất **`ngôn ngữ`**

In [ ]:
titles_synopses = data_df['Title'] + " " + data_df['Synopsis']
titles_synopses

In [ ]:
if not os.path.exists('titles_synopses_df.csv'):
  translator = Translator()
  translator.raise_Exception = True

  # Initilize the Google API translator
  new_titles_synopses = []
  for i, synopsis in enumerate(tqdm(titles_synopses)):
      # Using a sleep timer in order not to get timetout from google's API
      time.sleep(0.25)
      # Detect the language
      try:
        detection = translator.detect(synopsis)
        # If language is english with high confidence then don't translate
        if not ((detection.lang == "en")):
          translation = translator.translate(synopsis, dest="en")
          print(f"{translation.origin} ({translation.src}) --> {translation.text} ({translation.dest})")
          new_titles_synopses.append(translation.text)
        else:
          new_titles_synopses.append(synopsis)
      except Exception as e:
          # print(e, "for document", i)
          new_titles_synopses.append(synopsis)
          
  # Calling DataFrame constructor on list
  new_titles_synopses_df = pd.DataFrame(new_titles_synopses)
  new_titles_synopses_df.to_csv('titles_synopses_df.csv', index=False)

else:
  # Else load the translated text from the .csv file
  translated_titles_synopses_df = pd.read_csv('titles_synopses_df.csv')
  translated_titles_synopses_df = translated_titles_synopses_df.rename(columns={'0': 'translated_titles_synopses'})
  print("\n Final Translated text: ")
  print(translated_titles_synopses_df)

In [ ]:
titles_synopses

###### Vấn đề **`tiền xử lý ngôn ngữ tự nhiên`**

Vì các data được gather / crawl từ các trang web => sẽ gặp một số **`lỗi định dạng khi get text từ các thẻ html`** -> cần processing

Ngoài ra, sẽ áp dụng **`quy trình xử lý ngôn ngữ tự nhiên cơ bản`** nhằm mục đích giữ lại text mang nhiều ý nghĩa nhất có thể:
  - Khoảng trắng ( str.strip )

  - Xử lý viết tắt trong tiếng Anh (**`contractions`**)

  - Xóa các kí tự dấu (X**`punctuation`**)
  
  - Xóa các từ phổ biến trong tiếng Anh (**`stopwords`**)

  - Khôi phục về từ gốc

In [ ]:
punctuation = string.punctuation
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = stopwords.words('english')
ps = nltk.PorterStemmer()
wl = nltk.WordNetLemmatizer()


# Remove html tags. Get a string as input and return the string without html tags.
def remove_htmltags(html):
    return BeautifulSoup(html).get_text()

def remove_spacebar(text):
    return text.strip()

# Expanding contractions
def expand_contractions(text):
    return contractions.fix(text)

def remove_punct(text):
    text = "".join([char for char in text if char not in punctuation])
    return text

def remove_stopwords(text):
    tokens = word_tokenize(text.lower())
    text = " ".join([word for word in tokens if word not in stop_words])
    return text 

# def stemming(text):
#     tokens = word_tokenize(text)
#     stemmed_text=" ".join([wl.lemmatize(word) for word in tokens])
#     return stemmed_text

def lemmatizing(text):
    tokens = word_tokenize(text)
    lemmatized_text=" ".join([wl.lemmatize(word) for word in tokens])
    return lemmatized_text

In [ ]:
preprossed_translated_titles_synopses = translated_titles_synopses_df['translated_titles_synopses'].copy()

# Applying text preprocessing
preprossed_translated_titles_synopses = preprossed_translated_titles_synopses.map(lambda l: remove_htmltags(l))
preprossed_translated_titles_synopses = preprossed_translated_titles_synopses.map(lambda l: remove_spacebar(l))
preprossed_translated_titles_synopses = preprossed_translated_titles_synopses.map(lambda l: remove_punct(l))
preprossed_translated_titles_synopses = preprossed_translated_titles_synopses.map(lambda l: expand_contractions(l))
preprossed_translated_titles_synopses = preprossed_translated_titles_synopses.map(lambda l: remove_stopwords(l))
#preprossed_translated_titles_synopses = preprossed_translated_titles_synopses.map(lambda l: stemming(l))
preprossed_translated_titles_synopses = preprossed_translated_titles_synopses.map(lambda l: lemmatizing(l))

preprossed_translated_titles_synopses

##### **`Topic Modelling`** with **`LDA`** (Latent Dirichlet Allocation)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vec = TfidfVectorizer(use_idf=True, norm='l2',ngram_range=(1, 1), max_df=0.9, min_df=0.001)
tfidf_text = tfidf_vec.fit_transform(preprossed_translated_titles_synopses)
print('TF-IDF output shape:', tfidf_text.shape)

# n_components is the number of topics
lda_model = LatentDirichletAllocation(n_components=25, random_state=random_state)
lda_top = lda_model.fit_transform(tfidf_text)
print(lda_top.shape) 
print('LDA output shape:', lda_top.shape) # (no_of_doc,no_of_topics)
print("Final perplexity score on document set: ", lda_model.bound_)

**`Xem thử tỉ lệ các topic mà cuốn sách đầu tiên`** có thể thuộc về

In [ ]:
# Composition of doc 0
print("Document 0: ")
for i,topic in enumerate(lda_top[0]):
  print("Topic ",i,": ",topic*100,"%")

In [ ]:
# Most likely topic for all docs
for i, doc in enumerate(lda_top):
    print("Document:", i)
    print("Most likely topic: ", np.argmax(doc), ": ",max(doc)*100,"%" )

In [ ]:
# most important words for each topic
vocab = tfidf_vec.get_feature_names_out()

for i, comp in enumerate(lda_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

**Lưu các topics** thành DataFrame mới

In [ ]:
topics_df = pd.DataFrame(lda_top, columns=["Topic " + i.__str__() for i in range(lda_top.shape[1])])
topics_df

##### Cột **`Edition`**

###### Giải thích

**`Edition`** đang có xu hướng gộp dữ liệu, ta có thể preprocessing để tách dữ liệu và tạo feature mới => Phục vụ việc **`Visualization`**

**`Ví dụ: `**
- **`Paperback,– 10 Mar 2016`** thì: **'Paperback'** là tòa soạn, **'10 Mar 2016'** là ngày tháng và năm

In [ ]:
data_df.head(3)


Một số sample có thêm kiểu phiên bản như: Import, Illustrated, Special Edition, Student Edition                                
- **`Paperback,– Special Edition, 6 May 2008`** thì: **'Special Edition'** là kiểu phiên bản

In [ ]:
data_df[data_df['Edition'].str.contains("Special Edition")].head(3)

Ngoài ra, ở một số ít sample **'Ngày'**, **'Tháng'**, **'Năm'** có thể bị miss => cần handle và xử lý.

Ta sẽ cần split ',' chuỗi để lấy từng thành phần

Khi split ',' sẽ chưa chính xác vì có 5 sample chứa quốc gia xuất bản đứng đầu như: (German), (Spanish) => Vì số lượng quá nhỏ nên ta loại bỏ luôn để đồng nhất dữ liệu

In [ ]:
# Splits the language string-text based on comma in two new columns df
split_edition_df = data_df["Edition"].str.split(",", n = 1, expand = True)

# Marking the rows which contain the language property-tag in them
language_list = []
for item in split_edition_df[0]:
    if '(' in item and ')' in item:
        print(item)
        language_list.append(item)
    else:
        language_list.append('NA')
# Saving the marked rows in a new Pandas Series object        
language_series = pd.Series(language_list)

edition_with_removed_lang_series = data_df["Edition"].copy()
for i, element in enumerate(language_series):
  if element != 'NA':
    edition_with_removed_lang_series[i] = edition_with_removed_lang_series[i].replace(element+",", "", 1)

###### Tách **`Tòa soạn`**

In [ ]:
split_edition_df = edition_with_removed_lang_series.str.split(",", n = 1, expand = True)

print_series = split_edition_df[0]
print_series.unique()

###### Tách **`Năm Xuất Bản`**

In [ ]:
import string

# Define the function to remove the punctuations from the rows
def remove_punctuations(text):
    punctuations_to_remove = [punctuation for punctuation in string.punctuation if punctuation != ',']
    punctuations_to_remove.append('–')
    for punctuation in ['–']:
        text = text.replace(punctuation, '')
    return text

rest_edition_series = split_edition_df[1].apply(remove_punctuations)

print("Trước khi xóa")
print(split_edition_df[1][0:3])
print("")
print("Sau khi xóa")
print(rest_edition_series[0:3])

In [ ]:
def extract_year(text):
    text = text[-4:]
    return text

year_series = rest_edition_series.apply(extract_year)
year_series.unique()

Như ta thấy vẫn xuất hiện, những giá trị thuộc những sample sai cấu trúc => cho ra kết quả không phải **Năm**

In [ ]:
print('Tổng các sample không phải là năm: ', (len(year_series) - sum(year_series.str.isnumeric())))
year_series.loc[year_series.str.isnumeric() == False] = 'NA'
print("Sau khi process: ", year_series.unique())

###### Tách **`Tháng Xuất Bản`**

In [ ]:
temp_data = []

for i, row in enumerate(rest_edition_series):
  if year_series[i] != 'NA':
    temp_data.append(row[:-5])
  else:
    temp_data.append(row)

rest_edition_series = pd.Series(temp_data)

def extract_month(text):
    text = text[-3:]
    return text

month_series = rest_edition_series.apply(extract_month)
month_series
month_series.unique()

Ngoài 12 tháng ra, vẫn còn xuất hiện lỗi

In [ ]:
months = ['Apr','Aug','Dec','Feb', 'Jan', 'Jul','Jun','Mar','May','Nov','Oct','Sep']
for value in month_series:
  if value not in months:
    month_series = month_series.replace([value],'NA')

print(month_series.value_counts())

###### Tách **`Kiểu Phiên Bản`**

In [ ]:
temp_data = []

# Isolation of the rest of the text
for i, row in enumerate(rest_edition_series):
  if month_series[i] != 'NA':
    temp_data.append(row[:-4])
  else:
    temp_data.append(row)

rest_edition_series = pd.Series(temp_data)

In [ ]:
# Splits the rest of the text by the comma
rest_edition_list = [i.split(",") for i in list(rest_edition_series)]


no_day_lists = []
# Adding all the items of the rest of the text except the day of the month which was not included
for a_list in rest_edition_list:
    no_day_list = []
    for item in a_list:
        try:
            int(item)
        except ValueError:
            if item != '':
                no_day_list.append(item.strip())
    no_day_lists.append(no_day_list)    
    
# Taking the resultunt text and casting it to a Series object    
type_series = pd.Series(no_day_lists)
# Marking as 'NA_kind' the rows/books which do not include any kind of text as a type 
type_series = type_series.apply(lambda y: 'NA_kind' if (len(y)==0) or (y == [''])   
                                    else ','.join([elem.strip() for elem in y]))

### Kết thúc Preprocessing

#### Cập nhật datase ban đầu thành data preprocessing

In [ ]:
preprossed_data_df = data_df.copy()
preprossed_data_df.head(3)

In [ ]:

preprossed_data_df = preprossed_data_df.drop(['Title', 'Synopsis', 'Author', 'Edition'], axis=1)
#preprossed_data_df['Titles_synopses_translated'] = translated_titles_synopses_df['translated_titles_synopses'] # TODO PUT CLUSTERS INSTEAD
preprossed_data_df['Reviews'] = reviews
preprossed_data_df['Authors'] = authors
preprossed_data_df['No. Authors'] = num_of_authors
preprossed_data_df['Ratings'] = ratings

# Add features extracted from the edition column
preprossed_data_df['Print'] = print_series
preprossed_data_df['Type'] = type_series
preprossed_data_df['Month'] = month_series
preprossed_data_df['Year'] = year_series

# Add topic features
preprossed_data_df = pd.concat([preprossed_data_df, topics_df], axis=1)

preprossed_data_df

#### Data fill missing

##### Fill **`Năm bị thiếu`** (Dùng trung vị)

In [ ]:
preprossed_data_df['Year'] = preprossed_data_df['Year'].replace('NA', np.NaN)
preprossed_data_df['Year'].value_counts()

In [ ]:
print("Trước khi fill: ", preprossed_data_df.Year.isna().sum())
preprossed_data_df['Year'] = preprossed_data_df['Year'].fillna(preprossed_data_df['Year'].median())
print("Trước khi fill: ", preprossed_data_df.Year.isna().sum())

##### Fill **`Tháng bị thiếu`**

Random có trọng số

In [ ]:
preprossed_data_df['Month'] = preprossed_data_df['Month'].replace('NA', np.NaN)

In [ ]:
preprossed_data_df.Month.isna().sum()

In [ ]:
preprossed_data_df.Month.value_counts()

In [ ]:
# Probbabilities for each month
probs = preprossed_data_df.Month.value_counts(normalize=True)

preprossed_data_df.loc[preprossed_data_df.Month.isna(), 'Month'] = np.random.choice(probs.index, p=probs.values, 
                                                                   size=preprossed_data_df.Month.isna().sum())

In [ ]:
preprossed_data_df.Month.isna().sum()

In [ ]:
preprossed_data_df.Month.value_counts()

## Cyclical Encoding for Month

Since, month is a temporal feature which is categorized and cyclical feature, a specific encoding was implemented.  Cyclical features such as days or months of the year are treated as cyclical in the sense that their values display a cyclical pattern and are encoded as polar coordinates.

With the polar representation, it was possible to assign different values for every moment in time while also hold on to the cyclical similarities and differences. After the months are enoded to numbers (1 to 12), the formulas for the cyclical-polar encoding, which two new features will arise, are the following:

$$ month_{\cos }=\cos \left(\frac{2 \pi \times month}{\max (month)}\right)$$

$$ month_{\sin }=\sin \left(\frac{2 \pi \times  month}{\max (month)}\right) $$



In [ ]:
def month_to_int(df):
    months = ['Jan', 'Feb', 'Mar', 'Apr','May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    df['Month_Int'] = df['Month']
    
    for i, month in enumerate(months):
        df['Month_Int'] = df['Month_Int'].replace(month, i+1)
        
    return pd.to_numeric(df["Month_Int"], downcast="float")
    
preprossed_data_df['Month_Int'] = month_to_int(preprossed_data_df)

# Normalize x values to match with the 0-2π cycle
preprossed_data_df["Month_Norm"] = 2 * np.pi * preprossed_data_df["Month_Int"] / preprossed_data_df["Month_Int"].max()
# Cos and sin features
preprossed_data_df["Cos_Month"] = np.cos(preprossed_data_df["Month_Norm"])
preprossed_data_df["Sin_Month"] = np.sin(preprossed_data_df["Month_Norm"])
# preprossed_data_df["CONFIRM"] = preprossed_data_df["Cos_Month"]**2 + preprossed_data_df["Sin_Month"]**2 # SHOULD BE ONE

# Plotting months in a cyclical enocding
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(y=preprossed_data_df.Cos_Month, x=preprossed_data_df.Sin_Month, mode="markers"))

fig.update_layout(yaxis = dict(title="Cos_Month"),
                  xaxis = dict(title="Sin_Month", scaleanchor = "x", scaleratio = 1))
fig.show()

## Exporting data for visualization and modelling

In [ ]:
display(preprossed_data_df.info())

###### Lưu dữ liệu đã preprocessing để **`visualization`** và **`xây dựng model`**

In [ ]:
# Move target variable to the end
preprossed_data_df = preprossed_data_df[[c for c in preprossed_data_df if c not in ['Price']] 
                  + ['Price']]

In [ ]:
finalized_data_df =  preprossed_data_df.drop(['Month', 'Month_Int', 'Month_Norm'], axis=1)
visualization_data_df =  preprossed_data_df.drop(['Month_Int', 'Month_Norm'], axis=1)

In [ ]:
# Saving-exporting to new .csv file
finalized_data_df.to_csv('finalized_data_df.csv', index=False)
visualization_data_df.to_csv('visualization_data_df.csv', index=False)